# Test the 2020 AERR emission quantity sensitivity b/w old and new VMT Mix
Created by: Apoorb
--

## Housekeeping

In [59]:
import sys

sys.path.append(r"C:\Users\a-bibeka\PycharmProjects\FY23_VMT_Mix")

In [60]:
import pathlib
from pathlib import Path
import pandas as pd
import geopandas as gpd
import numpy as np
from vmtmix_fy23.utils import path_county_shp

Set Paths

In [61]:
path_aerr20_csvs = Path(
    r"E:\Texas A&M Transportation Institute\TxDOT_TPP_Projects - Task 5.3 Activity Forecasting Factors\Data\fy23_vmt_mix\Input\aerr20_output"
)
path_sutft_act = Path.joinpath(path_aerr20_csvs, "mvs303_act_sutft.csv")
path_sutft_emis = Path.joinpath(path_aerr20_csvs, "mvs303_emis_sutft.csv")
path_vmtmix_fy22_13_19 = Path(
    r"E:\Texas A&M Transportation Institute\TxDOT_TPP_Projects - Task 5.3 Activity Forecasting Factors\Data\fy23_vmt_mix\output\fy22_fin_vmtmix_13_19_042022.csv"
)

path_vmtmix_fy23_13_21 = Path(
    r"E:\Texas A&M Transportation Institute\TxDOT_TPP_Projects - Task 5.3 Activity Forecasting Factors\Data\fy23_vmt_mix\output\fy23_fin_vmtmix_13_21_032023.csv"
)

Analysis FIPS

In [62]:
gdf_county = gpd.read_file(path_county_shp)
district_map = gdf_county[["TXDOT_DIST", "FIPS_ST_CN"]].astype(int)

hgb_fips = [48201, 48039, 48157, 48473, 48339, 48291, 48071, 48167]
san_fips = [48029, 48091, 48187, 48259, 48493]
hgb_san_fips = list(hgb_fips) + list(san_fips)
fips_254 = list(district_map.FIPS_ST_CN.unique())
analysis_fips = fips_254


district_map_anly = district_map.loc[
    district_map.FIPS_ST_CN.isin(analysis_fips)
].rename(columns={"FIPS_ST_CN": "fips", "TXDOT_DIST": "txdot_dist"})

## Process AERR20 SUT-FT Emission and Activity to get County-level VMT and Emission Rates

In [63]:
sutft_act = pd.read_csv(path_sutft_act)
suft_emis = pd.read_csv(path_sutft_emis)

### Get county-level VMT and filter to VMT activity.

In [64]:
sutft_vmt = sutft_act.loc[sutft_act.mvs_act_type == "vmt"]
sutft_vmt = sutft_vmt.rename(columns={"activity": "vmt"})
sutft_vmt["vmt_agg"] = sutft_vmt.groupby(
    [
        "county",
        "fips",
        "year_id",
        "road_type_id",
        "roadtype_lab",
        "activity_type_id",
        "mvs_act_type",
        "act_em_lab",
    ]
).vmt.transform(sum)
sutft_vmt = sutft_vmt.filter(
    items=[
        "county",
        "fips",
        "year_id",
        "road_type_id",
        "roadtype_lab",
        "activity_type_id",
        "mvs_act_type",
        "source_type_id",
        "fuel_type_id",
        "act_em_lab",
        "vmt",
        "vmt_agg",
    ]
)

### Back-calculate Emission On-raod Emission Rate.

In [65]:
suft_emis_1 = (
    suft_emis.loc[
        (suft_emis.process_lab.isin(["Running_Exh", "Crank_Run_Exh"]))
        & (suft_emis.road_type_id != 1)
    ]
    .groupby(
        [
            "county",
            "fips",
            "year_id",
            "road_type_id",
            "roadtype_lab",
            "pollutant_id",
            "short_name_mvs3",
            "sort_order_pol",
            "eis_poll_nei17",
            "short_name_nei17",
            "source_type_id",
            "fuel_type_id",
        ],
        as_index=False,
    )
    .emission_quant.sum()
)

In [66]:
suft_emis_vmt = suft_emis_1.merge(
    sutft_vmt,
    on=[
        "county",
        "fips",
        "year_id",
        "road_type_id",
        "roadtype_lab",
        "source_type_id",
        "fuel_type_id",
    ],
    how="left",
)

In [67]:
suft_emis_vmt_onroad = suft_emis_vmt.loc[
    lambda df: (df.road_type_id != 1) & (df.vmt != 0)
]
suft_emis_vmt_onroad["emission_rate_calc"] = (
    suft_emis_vmt_onroad.emission_quant / suft_emis_vmt_onroad.vmt
)

C:\Users\a-bibeka\AppData\Local\Temp\ipykernel_20840\1111270578.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suft_emis_vmt_onroad["emission_rate_calc"] = (


Filter to Analysis FIPS and get Dennis's VMT-Mix from the Data

In [68]:
suft_emis_vmt_onroad_anly = suft_emis_vmt_onroad.loc[
    lambda df: df.fips.isin(analysis_fips)
]
suft_emis_vmt_onroad_anly["vmt_mix_dp"] = (
    suft_emis_vmt_onroad_anly.vmt / suft_emis_vmt_onroad_anly.vmt_agg
)

## Load FY22 and F23 VMT-Mix Created by AxB

In [69]:
vmtmix_fy22_13_19 = pd.read_csv(path_vmtmix_fy22_13_19)
vmtmix_fy23_13_21 = pd.read_csv(path_vmtmix_fy23_13_21)

Process FY22 2013 -- 2019 VMT-Mix and add suffix.

In [70]:
vmtmix_fy22_13_19 = vmtmix_fy22_13_19.merge(district_map_anly, on="txdot_dist")
assert (
    set(vmtmix_fy22_13_19.fips.unique()).symmetric_difference(set(analysis_fips))
    == set()
)
vmtmix_fy22_13_19_filt = vmtmix_fy22_13_19.loc[
    lambda df: (df.dowagg == "Wkd")
    & (df.tod == "day")
    & (df.mvs_rdtype_nm != "ALL")
    & (df.yearID == 2020)
]
assert len(analysis_fips) * 24 * 4 == len(vmtmix_fy22_13_19_filt)
vmtmix_fy22_13_19_filt["road_type_id"] = vmtmix_fy22_13_19_filt.mvs_rdtype.astype(int)
vmtmix_fy22_13_19_filt = vmtmix_fy22_13_19_filt.filter(
    items=["fips", "district", "road_type_id", "sourceTypeID", "fuelTypeID", "vmt_mix"]
).rename(
    columns={
        "sourceTypeID": "source_type_id",
        "fuelTypeID": "fuel_type_id",
        "vmt_mix": "vmt_mix_fy22_13_19",
    }
)

C:\Users\a-bibeka\AppData\Local\Temp\ipykernel_20840\1720590678.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vmtmix_fy22_13_19_filt["road_type_id"] = vmtmix_fy22_13_19_filt.mvs_rdtype.astype(int)


Process FY23 2013 -- 2021 VMT-Mix and add suffix.

In [71]:
vmtmix_fy23_13_21 = vmtmix_fy23_13_21.merge(district_map_anly, on="txdot_dist")
assert (
    set(vmtmix_fy23_13_21.fips.unique()).symmetric_difference(set(analysis_fips))
    == set()
)
vmtmix_fy23_13_21_filt = vmtmix_fy23_13_21.loc[
    lambda df: (df.dowagg == "Wkd")
    & (df.tod == "day")
    & (df.mvs_rdtype_nm != "ALL")
    & (df.yearID == 2020)
]
assert len(analysis_fips) * 24 * 4 == len(vmtmix_fy23_13_21_filt)
vmtmix_fy23_13_21_filt["road_type_id"] = vmtmix_fy23_13_21_filt.mvs_rdtype.astype(int)
vmtmix_fy23_13_21_filt = vmtmix_fy23_13_21_filt.filter(
    items=["fips", "district", "road_type_id", "sourceTypeID", "fuelTypeID", "vmt_mix"]
).rename(
    columns={
        "sourceTypeID": "source_type_id",
        "fuelTypeID": "fuel_type_id",
        "vmt_mix": "vmt_mix_fy23_13_21",
    }
)

C:\Users\a-bibeka\AppData\Local\Temp\ipykernel_20840\1619829859.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vmtmix_fy23_13_21_filt["road_type_id"] = vmtmix_fy23_13_21_filt.mvs_rdtype.astype(int)


## Merge AERR 2020 Processed Data with New VMT-Mixes

In [72]:
vmtmix_fy23_13_21_filt.columns

Index(['fips', 'district', 'road_type_id', 'source_type_id', 'fuel_type_id',
       'vmt_mix_fy23_13_21'],
      dtype='object')

In [73]:
suft_emis_vmt_onroad_anly_1 = suft_emis_vmt_onroad_anly.merge(
    vmtmix_fy22_13_19_filt,
    on=["fips", "road_type_id", "source_type_id", "fuel_type_id"],
).merge(
    vmtmix_fy23_13_21_filt,
    on=["district", "fips", "road_type_id", "source_type_id", "fuel_type_id"],
)
suft_emis_vmt_onroad_anly_1["vmt_fy22"] = (
    suft_emis_vmt_onroad_anly_1.vmt_agg * suft_emis_vmt_onroad_anly_1.vmt_mix_fy22_13_19
)

suft_emis_vmt_onroad_anly_1["vmt_fy23"] = (
    suft_emis_vmt_onroad_anly_1.vmt_agg * suft_emis_vmt_onroad_anly_1.vmt_mix_fy23_13_21
)

suft_emis_vmt_onroad_anly_1["emission_quant_fy22"] = (
    suft_emis_vmt_onroad_anly_1.emission_rate_calc
    * suft_emis_vmt_onroad_anly_1.vmt_fy22
    / 365
)
suft_emis_vmt_onroad_anly_1["emission_quant_fy23"] = (
    suft_emis_vmt_onroad_anly_1.emission_rate_calc
    * suft_emis_vmt_onroad_anly_1.vmt_fy23
    / 365
)

suft_emis_vmt_onroad_anly_1["emission_quant_dp"] = (
    suft_emis_vmt_onroad_anly_1.emission_quant / 365
)

In [74]:
suft_emis_vmt_onroad_anly_1.columns

Index(['county', 'fips', 'year_id', 'road_type_id', 'roadtype_lab',
       'pollutant_id', 'short_name_mvs3', 'sort_order_pol', 'eis_poll_nei17',
       'short_name_nei17', 'source_type_id', 'fuel_type_id', 'emission_quant',
       'activity_type_id', 'mvs_act_type', 'act_em_lab', 'vmt', 'vmt_agg',
       'emission_rate_calc', 'vmt_mix_dp', 'district', 'vmt_mix_fy22_13_19',
       'vmt_mix_fy23_13_21', 'vmt_fy22', 'vmt_fy23', 'emission_quant_fy22',
       'emission_quant_fy23', 'emission_quant_dp'],
      dtype='object')

In [75]:
emis_comp_pol = (
    suft_emis_vmt_onroad_anly_1.loc[
        lambda df: df.eis_poll_nei17.isin(
            ["CO", "NH3", "SO2", "NOX", "VOC", "CO2", "PM10-PRI", "PM25-PRI", 71432]
        )
    ]
    .groupby(["short_name_mvs3", "county", "fips", "district"], as_index=False)
    .agg(
        vmt_mix_dp=("vmt_mix_dp", "sum"),
        vmt_mix_fy22=("vmt_mix_fy22_13_19", "sum"),
        vmt_mix_fy23=("vmt_mix_fy23_13_21", "sum"),
        vmt_dp=("vmt", "sum"),
        vmt_fy22=("vmt_fy22", "sum"),
        vmt_fy23=("vmt_fy23", "sum"),
        emission_quant_dp=("emission_quant_dp", "sum"),
        emission_quant_fy22=("emission_quant_fy22", "sum"),
        emission_quant_fy23=("emission_quant_fy23", "sum"),
    )
)
emis_comp_pol = emis_comp_pol.assign(
    vmt_mix_diff_fy22_dp=lambda df: df.vmt_mix_fy22 - df.vmt_mix_dp,
    vmt_mix_diff_fy23_dp=lambda df: df.vmt_mix_fy23 - df.vmt_mix_dp,
    vmt_mix_diff_fy23_fy22=lambda df: df.vmt_mix_fy23 - df.vmt_mix_fy22,
    vmt_diff_fy22_dp=lambda df: df.vmt_fy22 - df.vmt_mix_dp,
    vmt_diff_fy23_dp=lambda df: df.vmt_fy23 - df.vmt_dp,
    vmt_diff_fy23_fy22=lambda df: df.vmt_fy23 - df.vmt_fy22,
    vmt_perdiff_fy22_dp=lambda df: 100 * df.vmt_diff_fy22_dp / df.vmt_dp,
    vmt_perdiff_fy23_dp=lambda df: 100 * df.vmt_diff_fy23_dp / df.vmt_dp,
    vmt_perdiff_fy23_fy22=lambda df: 100 * df.vmt_diff_fy23_fy22 / df.vmt_fy22,
    emission_quant_diff_fy22_dp=lambda df: df.emission_quant_fy22
    - df.emission_quant_dp,
    emission_quant_diff_fy23_dp=lambda df: df.emission_quant_fy23
    - df.emission_quant_dp,
    emission_quant_diff_fy22_fy23=lambda df: df.emission_quant_fy23
    - df.emission_quant_fy22,
    emission_quant_perdiff_fy22_dp=lambda df: 100
    * df.emission_quant_diff_fy22_dp
    / df.emission_quant_dp,
    emission_quant_perdiff_fy23_dp=lambda df: 100
    * df.emission_quant_diff_fy23_dp
    / df.emission_quant_dp,
    emission_quant_perdiff_fy23_fy22=lambda df: 100
    * df.emission_quant_diff_fy22_fy23
    / df.emission_quant_fy22,
)

Ouput file

In [76]:
path_out = Path.joinpath(path_vmtmix_fy22_13_19.parent, "QC", "aerr_vmt_mix_impact.csv")
emis_comp_pol.to_csv(path_out)

In [77]:
pd.options.display.float_format = "{:,.5f}".format

In [78]:
emis_comp_pol_co = emis_comp_pol.loc[
    lambda df: (df.short_name_mvs3.isin(["NOx"]) & (df.fips == 48353))
][
    [
        "vmt_mix_dp",
        "vmt_mix_fy22",
        "vmt_mix_fy23",
        "vmt_dp",
        "vmt_fy22",
        "vmt_fy23",
        "emission_quant_dp",
        "emission_quant_fy22",
        "emission_quant_fy23",
    ]
].sum()
emis_comp_pol_co

vmt_mix_dp                      4.00000
vmt_mix_fy22                    3.99912
vmt_mix_fy23                    3.99912
vmt_dp                414,585,308.95400
vmt_fy22              414,494,172.95734
vmt_fy23              414,502,688.41387
emission_quant_dp               1.60332
emission_quant_fy22             1.73516
emission_quant_fy23             1.70968
dtype: float64

In [79]:
emis_comp_pol.fips.unique()

array([48001, 48003, 48005, 48007, 48009, 48011, 48013, 48015, 48017,
       48019, 48021, 48023, 48025, 48027, 48029, 48031, 48033, 48035,
       48037, 48039, 48041, 48043, 48045, 48047, 48049, 48051, 48053,
       48055, 48057, 48059, 48061, 48063, 48065, 48067, 48069, 48071,
       48073, 48075, 48077, 48079, 48081, 48083, 48085, 48087, 48089,
       48091, 48093, 48095, 48097, 48099, 48101, 48103, 48105, 48107,
       48109, 48111, 48113, 48115, 48123, 48117, 48119, 48121, 48125,
       48127, 48129, 48131, 48133, 48135, 48137, 48141, 48139, 48143,
       48145, 48147, 48149, 48151, 48153, 48155, 48157, 48159, 48161,
       48163, 48165, 48167, 48169, 48171, 48173, 48175, 48177, 48179,
       48181, 48183, 48185, 48187, 48189, 48191, 48193, 48195, 48197,
       48199, 48201, 48203, 48205, 48207, 48209, 48211, 48213, 48215,
       48217, 48219, 48221, 48223, 48225, 48227, 48229, 48231, 48233,
       48235, 48237, 48239, 48241, 48243, 48245, 48247, 48249, 48251,
       48253, 48255,